In [ ]:
import numpy as np
import matplotlib.pyplot as plt


L = 1.0       # Domain length
Nx = 101      # Number of grid points
T = 1.0       # Simulation time
Nt = 100      # Number of time steps
hL = 0.0      # Left boundary condition
hR = 1.0      # Right boundary condition


In [2]:
K = 1.0       # Hydraulic conductivity
S = 0.0       # Source term

# Define the homotopy parameter
p = 1.0

In [3]:
h0 = np.linspace(hL, hR, Nx)
theta0 = np.linspace(0, 1, Nx)
t = np.linspace(0, T, Nt)
dt = T/Nt

In [4]:
dt

0.01

In [ ]:
k = 0

# Define the tolerance for convergence
tol = 1e-6

# Iterate until convergence
while p > tol and k < 10000:
    # Define the linear operator for the theta form
    A_theta = np.zeros((Nx, Nx))
    A_theta[0, 0] = 1
    A_theta[Nx-1, Nx-1] = 1
    for i in range(1, Nx-1):
        A_theta[i, i-1] = 1
        A_theta[i, i] = -2 - p*dt*K/(h0[i+1] - h0[i-1])
        A_theta[i, i+1] = 1
    b_theta = np.zeros(Nx)
    b_theta[0] = hL
    b_theta[Nx-1] = hR
    

In [ ]:
A_head = np.zeros((Nx, Nx))
    A_head[0, 0] = 1
    A_head[Nx-1, Nx-1] = 1
    
    for i in range(1, Nx-1):
        A_head[i, i-1] = -K*dt/(h0[i] - h0[i-1])
        A_head[i, i] = K*dt/(h0[i] - h0[i-1]) + K*dt/(h0[i+1] - h0[i-1])
        A_head[i, i+1] = -K*dt/(h0[i+1] - h0[i])
    b_head = np.zeros(Nx)
    b_head[0] = hL
    b_head[Nx-1] = hR

    

In [ ]:
 def F_theta(h):
        q = -K/(h[1:] - h[:-1])*(h[1:] - h[:-1])/2
        divq = np.zeros(Nx)
        divq[1:-1] = (q[1:] - q[:-1])/(h[1:] - h[:-1])
        return (h - h0)/dt + divq


In [ ]:
 # Define the Jacobian for the theta form
def J_theta(h):
    q = -K/(h[1:] - h[:-1])*(h[1:] - h[:-1])/2
    ddivq = np.zeros((Nx, Nx))
    for i in range(1, Nx-1):
        ddivq[i, i-1] = (K/(h[i] - h[i-1]))**2 - K/(h[i] - h[i-1])*(1 - K*(h[i+1] - h[i])/(h[i] - h[i-1])**2/(h[i+1] - h[i]))
        ddivq[i, i] = -(K/(h[i] - h[i-1]))**2 - (K/(h[i+1] - h[i]))**2 - K/(h[i] - h[i-1])*(1 + K*(h[i+1] - h[i])/(h[i] - h[i-1])**2/(h[i+1] - h[i])) + K*(h[i+1] - h[i])/(h[i] - h[i-1])**2/(h[i+1] - h[i])**2
        ddivq[i, i+1] = (K/(h[i+1] - h[i]))**2 + K/(h[i+1] - h[i])*(1 + K*(h[i+1] - h[i])/(h[i] - h[i-1])**2/(h[i+1] - h[i]))
    dFdh = np.eye(Nx) - p*dt*ddivq
    return dFdh
